## Actividad 2: Comparación de algoritmos (Regresión)

### Importación de librerías
Importamos las librerías que se van a usar a lo largo del ejercicio

In [ ]:
# Manejo de datos
from pandas import read_csv,DataFrame
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#Gráficos
import matplotlib.pyplot as plt
import seaborn as sns

#train-test
from sklearn.model_selection import train_test_split
from sklearn import model_selection

#metricas
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

#Algoritmos
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Descripción del dataset
Para este ejercicio se usará el dataset **Medical Cost Personal** extraído desde la página de Kaggle. El dataset cuenta con **1338** registros de personas y **7 atributos** a través de los cuales se intenta predecir el monto que les corresponde pagar por los gastos de se seguro de salud.

In [ ]:
#Leemos archivo desde una url de Kaggle
url_kaggle = (r'https://raw.githubusercontent.com/gad1989/TIA_actividad2/master/insurance.csv')
datos = read_csv(url_kaggle,delimiter=",")

### Caracteristicas del dataset
Realizamos una descripcion del dataset. Primero damos un primer vistazo a algunos registros

In [ ]:
datos.head(5)

Con la función shape, comprobamos que cuenta con **1338 registros** y **7 atributos (incluyendo el atributo a predecir)**

In [ ]:
print(datos.shape)

Visualizamos los tipos de datos de cada variable

In [ ]:
datos.dtypes

Visualizamos los diagramas de dispersión para las variables continuas para identificar si existe correlación entre ellas . En la diagonal vemos la forma de las distribuciones de cada una de las variables.

In [ ]:
sns.pairplot(datos[["charges","age", "bmi", "children"]], diag_kind="kde")

Para el caso de las variables categóricas, mostramos la frecuencia de cada una.

In [ ]:
datos.groupby("sex").size()

In [ ]:
datos.groupby("smoker").size()

In [ ]:
datos.groupby("region").size()

### Preparación de datos
Para proceder con los algoritmos de regresión se procede a convertir las variables vategóricas en numéricas, para ello se definen valores discretos para cada categoría de acuerdo al siguiente diccionario:

In [ ]:
convertion_dic = {"sex": {"female": 1, "male": 2},
                 "smoker": {"no": 0, "yes": 1},
                 "region": {"southeast": 1, "southwest": 2,"northeast":3,"northwest":4}}

In [ ]:
#Nuevo dataframe con atributos convertidos
datos = datos.replace(convertion_dic)
datos.head()

Separamos los atributos predictores y el atributo a predecir

In [ ]:
X = datos.drop("charges",axis=1)
y = datos["charges"]

Dividimos el dataset en 2: un dataset para entrenamiento y otro para test

In [ ]:
porcentaje = 0.25
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=porcentaje,random_state=1)

Ya que vamos a trabajar con redes neuronales es una buena práctica normalizar el dataset. Para ello usamos la función **min_max_scaler** que nos devuelve valores entre 0 y 1

In [ ]:
min_max_scaler = MinMaxScaler()
x_train_normalized = min_max_scaler.fit_transform(x_train)
x_test_normalized = min_max_scaler.fit_transform(x_test)

In [ ]:
DataFrame(x_train_normalized,columns=["age","sex","bmi","children","smoker","region"])

### Entrenamiento y validación de modelos
#### Modelo 1: Random Forest para regresión

Definimos los valores de los hiperparámetros y construimos el modelo

In [ ]:
n_estimators = 90
max_depth=5
criterion = "mae"

model = RandomForestRegressor(n_estimators=n_estimators,criterion=criterion,max_depth=max_depth,random_state=1)

Realizamos el entrenamiento del modelo con los datos normalizados

In [ ]:
model.fit(x_train_normalized, y_train)

Validamos el modelo usando los datos de test

In [ ]:
y_pred=model.predict(x_test_normalized)

Finalmente, se calculan las principales métricas de evaluación para la regresión

In [ ]:
print("RESULTADOS DE RANDOM FOREST")
#Error absoluto medio (MAE)
print("MAE: " + str(mean_absolute_error(y_test,y_pred)))

#Error cuadrático medio (MSE)
print("RMSE: " + str(np.sqrt(mean_squared_error(y_test,y_pred))))

#Coeficiente de determinación
print("Coeficiente de determinación: " + str(r2_score(y_test,y_pred)))                                    

#### Modelo 2: Redes Neuronales

Definimos los valores de los hiperparámetros:

In [ ]:
epochs = 1200
nodos_capa1 = 60
nodos_capa2 = 60
nodos_capa3 = 60
f_act='relu'
loss='mae'

Construimos el modelo de redes neuronales con 3 capas profundas y una capa de salida

In [ ]:
#Configura semilla aleatoria
tf.random.set_seed(1)

def build_model():
  modelo = keras.Sequential([
    layers.Dense(nodos_capa1, activation=f_act,input_shape=[6]),
    layers.Dense(nodos_capa2, activation=f_act),
    layers.Dense(nodos_capa3, activation=f_act),
    layers.Dense(1)
  ])

  modelo.compile(loss=loss,
                optimizer='adam',
                metrics=['mae', 'mse'])
  return modelo

In [ ]:
modelRN = build_model()

Realizamos una pequeña validación para comprobar la correcta construcción del modelo

In [ ]:
example_batch = x_train_normalized[:10]
example_result = modelRN.predict(example_batch)
example_result

Descripción del modelo

In [ ]:
modelRN.summary()

Entrenamos el modelo

In [ ]:
history = modelRN.fit(x_train_normalized, y_train,epochs=epochs, validation_split = 0.25, verbose=0)

Visualizamos el progreso del entrenamiento que se guardó en la variable "history"

In [ ]:
hist = DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

Graficamos la evolución del MAE con los datos de entrenamiento

In [ ]:
DataFrame(history.history)["mae"].plot(label='Train Error')
DataFrame(history.history)["val_mae"].plot(label='Validation Error')
plt.ylabel("mae")
plt.xlabel("epochs")
plt.legend()

In [ ]:
#Metricas para el conjunto de test
loss, mae, mse = modelRN.evaluate(x_test_normalized, y_test, verbose=2)
print()

Validamos el modelo usando los datos de test y calculamos las métricas

In [ ]:
test_predictions = modelRN.predict(x_test_normalized).flatten()

In [ ]:
print("RESULTADOS DE REDES NEURONALES")
#Error absoluto medio (MAE)
print("MAE: " + str(mean_absolute_error(y_test,test_predictions)))

#Error cuadrático medio (MSE)
print("RMSE: " + str(np.sqrt(mean_squared_error(y_test,test_predictions))))

#Coeficiente de determinación
print("Coeficiente de determinación: " + str(r2_score(y_test,test_predictions)))     

Mostramos los resultados de ambos algoritmos de forma gráfica

In [ ]:
plt.scatter(y_test, test_predictions,label='Redes Neuronales')
plt.scatter(y_test, y_pred,label='Random Forest')
plt.legend()
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
plt.plot([-100, 60000], [-100, 60000])